<a href="https://colab.research.google.com/github/araujobma/DesafioAluraGoogleGemini/blob/main/DesafioAlura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Uso de LLM para busca de produtos através de agentes**

### A ideia central por trás dos agentes é usar um modelo de linguagem para escolher uma sequência de ações a serem tomadas. Em agentes, um modelo de linguagem é usado como um mecanismo de raciocínio para determinar quais ações tomar e em que ordem.

Além da biblioteca google-generativeai importaremos framework langchain que nos permitirar criar agentes baseados na LLM Gemini(https://python.langchain.com/v0.1/docs/get_started/introduction/)



In [1]:
!pip install --upgrade --quiet  langchain langchainhub langchain-google-genai serpapi google-generativeai

Neste notebook iremos utilizar a serpapi(https://serpapi.com/) que permite o uso da pesquisa do google através de uma API. A API tem uma versão gratuita que
pode ser criada uma chave. Abaixo inicializamos os valores das api keys tanto do gemini quanto do serpapi

In [2]:
from google.colab import userdata
import os

#tanto o GEMINI_API_KEY e SERP_API_KEY devem estar definidos no Secrets do Colab
os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY")
os.environ["SERP_API_KEY"] = userdata.get("SERP_API_KEY")

In [15]:

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro")

Aqui importamos o que é necessário para criar um agente baseado na LLM Gemini.

In [16]:
from langchain.agents import AgentExecutor, tool, create_react_agent
from langchain import hub


## **Definição das tools que o agente terá acesso**
### Com base na nossa pergunta ao Agente, ele definirá a necessidade de usar as tools que disponibilizaremos a ele.

Abaixo definimos funções que fazem busca no google através da serpapi e "ler" uma página html como texto. Dada a definição das tools(funções), o agente vai saber como utilizá-las adequadamente.

In [17]:
from langchain.document_loaders import WebBaseLoader
import serpapi

@tool
def google_search(query: str) -> dict:
    """
    Retorna resultados da pesquisa do google baseada
    no parâmetro query
    """


    params = {
      "q": query,
      "hl": "pt-br",
      "gl": "br",
      "google_domain": "google.com",
      "api_key": os.environ["SERP_API_KEY"]
    }

    search = serpapi.search(params)
    shopping_results = search.get('shopping_results', [])
    organic_results = search.get('organic_results', [])

    if len(organic_results) == 0:
      return "No results. Try another query"

    return {"shopping_results": shopping_results, "organic_results": organic_results}


@tool
def get_url_content(url: str) -> str:
    """
    Retorna o texto de uma pagina html passando
    a url como parâmetro
    """

    try:
      loader = WebBaseLoader(url)

      docs = loader.load()
      return docs[0].page_content
    except:
      return "Nothing Found"


In [18]:
# default prompt for create_react_agent
prompt = hub.pull("hwchase17/react")
tools = [google_search, get_url_content]

agent= create_react_agent(
    llm,
    tools,
    prompt=prompt)

In [19]:
#Aqui deixamos o verbose=True para ir observando a linha de raciocínio do agente e para debugar.
    # O max_iterations é o máximo de ações que o agente vai realizar antes de parar.
agent_executor = AgentExecutor(
        agent=agent, tools=tools, handle_parsing_errors=True, verbose=True, max_iterations=15, max_execution_time=180
)

Agora definimos o que queremos que o agente pesquise.
Foram passadas algumas instruções gerais, mas é o agente que decidi o que ele vai fazer.
Aqui é um bom ponto pra brincar com esse input e ver o que o agente fará 😀

In [20]:
input = (
    """
    - Preciso pesquisar carros hatch vendidos no Brasil para decidir o que irei comprar.

    -retorne o resultado dos diferentes modelos encontrados formatado em um texto tabular com as colunas:
     ['Modelo', 'Marca', 'Motor', 'Tipo Transmissão', 'Ano de Fabricação', 'Preço', 'Pontos Fortes', 'Pontos fracos'].

    - Se alguma url estiver quebrada, pesquise em outra url

    - Tente usar o google_search o mínimo possível

   """
)
result = agent_executor.invoke({"input": input})



> Entering new AgentExecutor chain...
I need to find the different hatchback car models sold in Brazil and their specifications. I can use google_search to find a list of car models and then use get_url_content to get the specifications of each model.
Action: google_search
Action Input: "Carros hatch vendidos no Brasil"{'shopping_results': [], 'organic_results': [{'position': 1, 'title': 'Veja os 20 hatches mais vendidos do Brasil em 2023', 'link': 'https://autoesporte.globo.com/mercado/noticia/2024/01/veja-os-20-hatches-mais-vendidos-do-brasil-em-2023.ghtml', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://autoesporte.globo.com/mercado/noticia/2024/01/veja-os-20-hatches-mais-vendidos-do-brasil-em-2023.ghtml&ved=2ahUKEwiatdOC24SGAxXKpLAFHSAqDrkQFnoECBwQAQ', 'displayed_link': 'https://autoesporte.globo.com › noticia › 2024/01 › vej...', 'favicon': 'https://serpapi.com/searches/663eed4b5c1059b9bc36b3d3/images/d9bb654e9e15ae00d204aa0be0b90ad0a

In [21]:
print(result["output"])

| Modelo | Marca | Motor | Tipo Transmissão | Ano de Fabricação | Preço | Pontos Fortes | Pontos Fracos |
|---|---|---|---|---|---|---|---|
| Volkswagen Polo | Volkswagen | 1.0 MPI | Manual / Automática | 2023 | R$ 85.990 | Bom desempenho, baixo consumo de combustível, design moderno | Espaço interno limitado |
| Chevrolet Onix | Chevrolet | 1.0 Turbo | Manual / Automática | 2023 | R$ 72.990 | Bom desempenho, baixo consumo de combustível, boa relação custo-benefício | Espaço interno limitado |
| Hyundai HB20 | Hyundai | 1.0 Turbo | Manual / Automática | 2023 | R$ 79.990 | Bom desempenho, baixo consumo de combustível, boa relação custo-benefício | Espaço interno limitado |
| Fiat Mobi | Fiat | 1.0 Fire | Manual | 2023 | R$ 60.990 | Baixo preço, baixo consumo de combustível | Desempenho fraco, acabamento simples |
| Fiat Argo | Fiat | 1.0 Fire | Manual / Automática | 2023 | R$ 74.990 | Bom espaço interno, boa relação custo-benefício | Desempenho fraco, acabamento simples |
| Renault Kwid

Agora pedimos pra o modelo formatar o csv corretamente:

In [23]:
csv_str = llm.invoke("""
retorne um csv separado por ';' viŕgula com o cabeçalho:
['Nome', 'Marca', 'Motor', 'Tipo Transmissão', 'Ano de Fabricação', 'Preço', 'Pontos Fortes', 'Pontos fracos']
e remova qualque extra '|' dado:
```
{}
```
""".format(result["output"]))
print(csv_str.content)

Nome;Marca;Motor;Tipo Transmissão;Ano de Fabricação;Preço;Pontos Fortes;Pontos Fracos
Volkswagen Polo;Volkswagen;1.0 MPI;Manual / Automática;2023;R$ 85.990;Bom desempenho, baixo consumo de combustível, design moderno;Espaço interno limitado
Chevrolet Onix;Chevrolet;1.0 Turbo;Manual / Automática;2023;R$ 72.990;Bom desempenho, baixo consumo de combustível, boa relação custo-benefício;Espaço interno limitado
Hyundai HB20;Hyundai;1.0 Turbo;Manual / Automática;2023;R$ 79.990;Bom desempenho, baixo consumo de combustível, boa relação custo-benefício;Espaço interno limitado
Fiat Mobi;Fiat;1.0 Fire;Manual;2023;R$ 60.990;Baixo preço, baixo consumo de combustível;Desempenho fraco, acabamento simples
Fiat Argo;Fiat;1.0 Fire;Manual / Automática;2023;R$ 74.990;Bom espaço interno, boa relação custo-benefício;Desempenho fraco, acabamento simples
Renault Kwid;Renault;1.0 SCe;Manual / Automática;2023;R$ 59.990;Baixo preço, baixo consumo de combustível;Desempenho fraco, acabamento simples
Peugeot 208;Peu

## Com os dados retornados do agente, podemos criar um dataframe e gerar embeddings para filtrar as carecteríscitcas que desejamos no carro 😺

In [32]:
import numpy as np
import pandas as pd

In [88]:
file_name = '/tmp/tmp-result.csv'
with open('/tmp/tmp-result.csv', 'w')as f:
  f.write(csv_str.content)

df = pd.read_csv(file_name, sep=';')
df

,Nome,Marca,Motor,Tipo Transmissão,Ano de Fabricação,Preço,Pontos Fortes,Pontos Fracos
0,Volkswagen Polo,Volkswagen,1.0 MPI,Manual / Automática,2023,R$ 85.990,"Bom desempenho, baixo consumo de combustível, ...",Espaço interno limitado
1,Chevrolet Onix,Chevrolet,1.0 Turbo,Manual / Automática,2023,R$ 72.990,"Bom desempenho, baixo consumo de combustível, ...",Espaço interno limitado
2,Hyundai HB20,Hyundai,1.0 Turbo,Manual / Automática,2023,R$ 79.990,"Bom desempenho, baixo consumo de combustível, ...",Espaço interno limitado
3,Fiat Mobi,Fiat,1.0 Fire,Manual,2023,R$ 60.990,"Baixo preço, baixo consumo de combustível","Desempenho fraco, acabamento simples"
4,Fiat Argo,Fiat,1.0 Fire,Manual / Automática,2023,R$ 74.990,"Bom espaço interno, boa relação custo-benefício","Desempenho fraco, acabamento simples"
5,Renault Kwid,Renault,1.0 SCe,Manual / Automática,2023,R$ 59.990,"Baixo preço, baixo consumo de combustível","Desempenho fraco, acabamento simples"
6,Peugeot 208,Peugeot,1.6 16V,Manual / Automática,2023,R$ 99.990,"Design moderno, bom desempenho","Preço elevado, espaço interno limitado"
7,Citroën C3,Citroën,1.6 16V,Manual / Automática,2023,R$ 84.990,"Design moderno, bom espaço interno","Desempenho fraco, acabamento simples"
8,Toyota Yaris,Toyota,1.5 Dual VVT-i,Automática,2023,R$ 104.990,"Confiabilidade, baixo consumo de combustível","Preço elevado, desempenho fraco"
9,Honda City,Honda,1.5 i-VTEC,Automática,2023,R$ 114.990,"Bom espaço interno, bom desempenho","Preço elevado, consumo de combustível elevado"


In [89]:
import google.generativeai as genai

def embed_fn(row, df_columns, model='models/embedding-001'):
  row_str = ''
  for col in df_columns[1:]:
     row_str += (' ' + col+ ' ' + str(row[col]))
  return genai.embed_content(model=model,
                             content=row_str,
                             title=row['Nome'],
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]


df["Embeddings"] = df.apply(lambda row: embed_fn(row, df.columns), axis=1)
df

,Nome,Marca,Motor,Tipo Transmissão,Ano de Fabricação,Preço,Pontos Fortes,Pontos Fracos,Embeddings
0,Volkswagen Polo,Volkswagen,1.0 MPI,Manual / Automática,2023,R$ 85.990,"Bom desempenho, baixo consumo de combustível, ...",Espaço interno limitado,"[0.018980937, -0.054647, -0.036113035, 0.02066..."
1,Chevrolet Onix,Chevrolet,1.0 Turbo,Manual / Automática,2023,R$ 72.990,"Bom desempenho, baixo consumo de combustível, ...",Espaço interno limitado,"[0.005480333, -0.033782266, -0.0647709, 0.0231..."
2,Hyundai HB20,Hyundai,1.0 Turbo,Manual / Automática,2023,R$ 79.990,"Bom desempenho, baixo consumo de combustível, ...",Espaço interno limitado,"[-0.010559612, -0.077970214, -0.068368845, 0.0..."
3,Fiat Mobi,Fiat,1.0 Fire,Manual,2023,R$ 60.990,"Baixo preço, baixo consumo de combustível","Desempenho fraco, acabamento simples","[0.02846044, -0.050480388, -0.051982384, 0.010..."
4,Fiat Argo,Fiat,1.0 Fire,Manual / Automática,2023,R$ 74.990,"Bom espaço interno, boa relação custo-benefício","Desempenho fraco, acabamento simples","[0.04559086, -0.04777453, -0.067528956, 0.0115..."
5,Renault Kwid,Renault,1.0 SCe,Manual / Automática,2023,R$ 59.990,"Baixo preço, baixo consumo de combustível","Desempenho fraco, acabamento simples","[0.029017983, -0.038847726, -0.06647424, -0.01..."
6,Peugeot 208,Peugeot,1.6 16V,Manual / Automática,2023,R$ 99.990,"Design moderno, bom desempenho","Preço elevado, espaço interno limitado","[0.04098653, -0.06587298, -0.06366229, 0.00648..."
7,Citroën C3,Citroën,1.6 16V,Manual / Automática,2023,R$ 84.990,"Design moderno, bom espaço interno","Desempenho fraco, acabamento simples","[0.021673732, -0.06932229, -0.06977724, -0.009..."
8,Toyota Yaris,Toyota,1.5 Dual VVT-i,Automática,2023,R$ 104.990,"Confiabilidade, baixo consumo de combustível","Preço elevado, desempenho fraco","[0.018859405, -0.06468766, -0.03128457, 0.0304..."
9,Honda City,Honda,1.5 i-VTEC,Automática,2023,R$ 114.990,"Bom espaço interno, bom desempenho","Preço elevado, consumo de combustível elevado","[0.02887058, -0.020745257, -0.027513389, 0.026..."


In [90]:
def gerar_e_buscar_consulta(consulta, base, model='models/embedding-001'):
  embedding_da_consulta = genai.embed_content(model=model,
                             content=consulta,
                             task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  # aqui retornamos os 3 resultados mais proximos da query
  indices = np.argsort(produtos_escalares)[-3:]

  return df.iloc[indices]

In [91]:
consulta = "carro com baixo consumo"

gerar_e_buscar_consulta(consulta, df)

,Nome,Marca,Motor,Tipo Transmissão,Ano de Fabricação,Preço,Pontos Fortes,Pontos Fracos,Embeddings
1,Chevrolet Onix,Chevrolet,1.0 Turbo,Manual / Automática,2023,R$ 72.990,"Bom desempenho, baixo consumo de combustível, ...",Espaço interno limitado,"[0.005480333, -0.033782266, -0.0647709, 0.0231..."
5,Renault Kwid,Renault,1.0 SCe,Manual / Automática,2023,R$ 59.990,"Baixo preço, baixo consumo de combustível","Desempenho fraco, acabamento simples","[0.029017983, -0.038847726, -0.06647424, -0.01..."
12,Volkswagen Gol,Volkswagen,1.0 MPI,Manual,2023,R$ 79.990,"Baixo preço, baixo consumo de combustível","Desempenho fraco, acabamento simples","[0.02592015, -0.047528222, -0.06240594, 0.0258..."


In [92]:
consulta = "Motor potente"

gerar_e_buscar_consulta(consulta, df)

,Nome,Marca,Motor,Tipo Transmissão,Ano de Fabricação,Preço,Pontos Fortes,Pontos Fracos,Embeddings
13,Mini Cooper,Mini,1.5 TwinPower Turbo,Automática,2023,R$ 229.990,"Design icônico, bom desempenho","Preço elevado, espaço interno limitado","[0.015970135, -0.039598327, -0.048334464, 0.01..."
14,GWM Ora 03,GWM,Elétrico,Automática,2023,R$ 159.990,"Baixo custo de manutenção, bom desempenho","Preço elevado, autonomia limitada","[0.029002942, -0.056649633, -0.03503853, 0.016..."
3,Fiat Mobi,Fiat,1.0 Fire,Manual,2023,R$ 60.990,"Baixo preço, baixo consumo de combustível","Desempenho fraco, acabamento simples","[0.02846044, -0.050480388, -0.051982384, 0.010..."


In [93]:
consulta = "bom espaço interno"

gerar_e_buscar_consulta(consulta, df)

,Nome,Marca,Motor,Tipo Transmissão,Ano de Fabricação,Preço,Pontos Fortes,Pontos Fracos,Embeddings
6,Peugeot 208,Peugeot,1.6 16V,Manual / Automática,2023,R$ 99.990,"Design moderno, bom desempenho","Preço elevado, espaço interno limitado","[0.04098653, -0.06587298, -0.06366229, 0.00648..."
4,Fiat Argo,Fiat,1.0 Fire,Manual / Automática,2023,R$ 74.990,"Bom espaço interno, boa relação custo-benefício","Desempenho fraco, acabamento simples","[0.04559086, -0.04777453, -0.067528956, 0.0115..."
10,Renault Stepway,Renault,1.6 SCe,Manual / Automática,2023,R$ 87.990,"Bom espaço interno, boa relação custo-benefício","Desempenho fraco, acabamento simples","[0.045985825, -0.05304749, -0.06016072, -0.010..."


## Usando o Terminal interativo python para responder uma pergunta.

Neste caso é fornecido ao agente um terminal python pra ele usar e responder à nossa pergunta:

In [102]:
!pip install -q langchain-experimental
from langchain.agents import create_react_agent
from langchain_experimental.tools import PythonAstREPLTool

In [113]:
instructions = """Você é um agente projetado para escrever e executar código Python para responder a perguntas. Você tem acesso a um Python_REPL,
 que você pode usar para executar código Python. Se você receber um erro, depure seu código e tente novamente. Use apenas a saída do seu código
 para responder à pergunta. Você pode saber a resposta sem executar nenhum código, mas ainda deve executar o código para obter a resposta.
 Se parecer que você não consegue escrever código para responder à pergunta, basta retornar "Eu não sei" como resposta.
"""
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)

In [114]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.0)
tools = [PythonAstREPLTool()]
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)


In [115]:
resposta = agent_executor.invoke({"input": """
Dado o seguinte csv: {}

Lembre-se de transformar 99.999 em um float 99999.00

e filtre o que tem Preço < 100000
""".format(csv_str.content)})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: python_repl_ast
Action Input: df = pd.read_csv('cars.csv', sep=';')NameError: name 'pd' is not definedDo I need to use a tool? Yes
Action: python_repl_ast
Action Input: import pandas as pdDo I need to use a tool? Yes
Action: python_repl_ast
Action Input: df = pd.read_csv('cars.csv', sep=';')Do I need to use a tool? Yes
Action: python_repl_ast
Action Input: df = df[df['Preço'] < 100000]TypeError: '<' not supported between instances of 'str' and 'int'Do I need to use a tool? Yes
Action: python_repl_ast
Action Input: df['Preço'] = df['Preço'].str.replace('R$', '').str.replace('.', '').astype(float)Do I need to use a tool? Yes
Action: python_repl_ast
Action Input: df = df[df['Preço'] < 100000]Do I need to use a tool? No
Final Answer: Chevrolet Onix, Hyundai HB20, Fiat Mobi, Fiat Argo, Renault Kwid, Volkswagen Gol

> Finished chain.


In [110]:
resposta["output"]

'Chevrolet Onix\nFiat Mobi\nFiat Argo\nRenault Kwid\nVolkswagen Gol'

## **Conlusão**

### Podemos dividir nossa tarefa em várias partes e usar uma LLM pra cada. Desse modo cada etapa é executada por um agente diferente e conseguimos obter melhores resultados

### Sugestão: tente mudar o notebook para pesquisar outro tipo de produto ou usar outras ferramentas(tools)
